In [36]:
import pandas as pd
import graphistry

class HyperNetXG:
  
    def __init__(self, graphistry):
        self.graphistry = graphistry      

    def hypernetx_to_graphistry_bipartite(self, H):

        nodes_df = pd.concat(
            [pd.DataFrame({
                'node': [x["node"] for x in H["nodes"]], 
                'type': 'hypernode'}),
             pd.DataFrame({
                 'node': [x["edge"] for x in H["edges"]], 
                 'type': 'hyperedge'})],
            ignore_index=True,
            sort=False)

        edges_df = pd.concat(
            [ pd.DataFrame({'src': [], 'dst': []}) ] + 
            [
                pd.DataFrame({
                    'src': k,
                    'dst': [x["node"] for x in H["incidences"] if x["edge"] == k]
                })
                for k in {x["edge"] for x in H["incidences"]}
            ], ignore_index=True, sort=False)

        return self.graphistry.bind(
              source='src', 
              destination='dst', 
              node='node').nodes(nodes_df).edges(edges_df)


hg = HyperNetXG(graphistry)

In [33]:
import json

with open("e-coli.json") as file:
    H = json.load(file)

In [37]:
g = hg.hypernetx_to_graphistry_bipartite(H)
g._nodes.sample(3)

,node,type
57,h2o_c,hypernode
24,gln__L_e,hypernode
174,TKT2,hyperedge


In [38]:
g._edges.sample(3)

,src,dst
404,PIt2r,pi_c
405,CO2t_reverse,co2_c
493,ENO_reverse,2pg_c
